In [ ]:
!pip install nudenet ffmpeg-python #Make sure ffmpeg is installed on your system globally

In [ ]:
# Print all classes threshold 
import ffmpeg
import cv2
from nudenet import NudeDetector
from datetime import timedelta

# Initialize the NudeNet detector
detector = NudeDetector()

# Paths
input_video = "/workspace/GoT/5.mp4"
converted_video = "converted_video.mp4"

# Convert to H.265 format
ffmpeg.input(input_video).output(converted_video, vcodec='libx265', crf=28).run()

# Open the converted video using OpenCV
cap = cv2.VideoCapture(converted_video)

# Get the frame rate to determine timestamps
fps = cap.get(cv2.CAP_PROP_FPS)
frame_interval = int(fps)  # Process 1 frame per second

nudity_timestamps = []  # List to store nudity detections

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Only process frames at the defined interval
    if frame_count % frame_interval == 0:
        # Detect nudity on the frame
        detections = detector.detect(frame)

        # Capture timestamp and all detections in that frame
        timestamp = timedelta(seconds=frame_count // fps)
        frame_detections = [f"{d['class']} ({d['score']:.2f})" for d in detections]
        
        # Only store frames with at least one detection
        if frame_detections:
            nudity_timestamps.append(f"{str(timestamp)}: " + ", ".join(frame_detections))

    frame_count += 1

cap.release()

# Output the results
print("This video contains nudity. Press 'i' for details.")
input("Press 'i' to view timestamps: ")
for entry in nudity_timestamps:
    print(entry)


In [ ]:
# censoring at only 1 sec frame
import ffmpeg
import cv2
from nudenet import NudeDetector
from datetime import timedelta
import os

# Initialize the NudeNet detector
detector = NudeDetector()

# Paths
input_video = "/workspace/GoT/2.mp4"
converted_video = "converted_video.mp4"
censored_output_video = "censored_video.mp4"
temp_image_path = "temp_frame.jpg"  # Temporary file for each frame
temp_censored_image_path = "temp_censored_frame.jpg"  # Temporary censored image path

# Convert to H.265 format with overwriting permission
ffmpeg.input(input_video).output(converted_video, vcodec='libx265', crf=28).overwrite_output().run()

# Open the converted video using OpenCV
cap = cv2.VideoCapture(converted_video)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_interval = int(fps)  # Process 1 frame per second

# Define video writer for the censored video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(censored_output_video, fourcc, fps, (width, height))

# List to store nudity timestamps
nudity_timestamps = []

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Only process frames at the defined interval
    if frame_count % frame_interval == 0:
        # Save the frame as a temporary image file
        cv2.imwrite(temp_image_path, frame)

        # Detect nudity in the saved image
        detections = detector.detect(temp_image_path)

        # If any nudity is detected, apply censoring and log details
        if detections:
            # Censor the frame
            detector.censor(temp_image_path, output_path=temp_censored_image_path)

            # Load the censored frame back into memory
            censored_frame = cv2.imread(temp_censored_image_path)

            # Capture the timestamp and classes with scores
            timestamp = timedelta(seconds=frame_count // fps)
            frame_detections = [f"{d['class']} ({d['score']:.2f})" for d in detections]
            nudity_timestamps.append(f"{str(timestamp)}: " + ", ".join(frame_detections))

            # Use the censored frame
            frame = censored_frame

    # Write frame (censored or not) to output video
    out.write(frame)
    frame_count += 1

cap.release()
out.release()

# Remove the temporary image files
if os.path.exists(temp_image_path):
    os.remove(temp_image_path)
if os.path.exists(temp_censored_image_path):
    os.remove(temp_censored_image_path)

# Output the results
print("This video contains nudity. Press 'i' for details.")
input("Press 'i' to view timestamps: ")
for entry in nudity_timestamps:
    print(entry)

print(f"Censored video saved as {censored_output_video}")


In [ ]:
# Final Code

import ffmpeg
import cv2
from nudenet import NudeDetector
from datetime import timedelta
import os

# Initialize the NudeNet detector
detector = NudeDetector()

# Paths
input_video = "/workspace/GoT/2.mp4"
converted_video = "converted_video.mp4"
censored_output_video = "censored_video.mp4"
temp_image_path = "temp_frame.jpg"  # Temporary file for each frame
temp_censored_image_path = "temp_censored_frame.jpg"  # Temporary censored image path

# Convert to H.265 format with overwriting permission
ffmpeg.input(input_video).output(converted_video, vcodec='libx265', crf=28).overwrite_output().run()

# Open the converted video using OpenCV
cap = cv2.VideoCapture(converted_video)
fps = cap.get(cv2.CAP_PROP_FPS)  # Get the original frame rate
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define video writer for the censored video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(censored_output_video, fourcc, fps, (width, height))

# List to store nudity timestamps
nudity_timestamps = []

# List of classes to ignore
ignored_classes = {
    "FEMALE_GENITALIA_COVERED",
    "FACE_FEMALE",
    "BELLY_COVERED",
    "FEET_COVERED",
    "ARMPITS_COVERED",
    "ARMPITS_EXPOSED",
    "BELLY_EXPOSED",
    "ANUS_COVERED",
    "FEMALE_BREAST_COVERED",
    "BUTTOCKS_COVERED",
    "FACE_MALE"
}

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Save the frame as a temporary image file
    cv2.imwrite(temp_image_path, frame)

    # Detect nudity in the saved image
    detections = detector.detect(temp_image_path)

    # Filter detections to exclude ignored classes
    valid_detections = [d for d in detections if d['class'] not in ignored_classes]

    # Sort detections by score, descending order
    valid_detections = sorted(valid_detections, key=lambda x: x['score'], reverse=True)

    # If there are valid detections, apply censoring and log details
    if valid_detections and valid_detections[0]['score'] >= 0.25:  # Threshold check
        # Capture the timestamp and classes with scores
        timestamp = timedelta(seconds=frame_count // fps)
        frame_detections = [f"{d['class']} ({d['score']:.2f})" for d in valid_detections]
        nudity_timestamps.append(f"{str(timestamp)}: " + ", ".join(frame_detections))

        # Censor the frame by applying censoring only to valid detections
        detector.censor(temp_image_path, output_path=temp_censored_image_path)

        # Load the censored frame back into memory
        censored_frame = cv2.imread(temp_censored_image_path)

        # Use the censored frame
        frame = censored_frame

    # Write the frame (censored or not) to the output video
    out.write(frame)
    frame_count += 1

cap.release()
out.release()

# Remove the temporary image files
if os.path.exists(temp_image_path):
    os.remove(temp_image_path)
if os.path.exists(temp_censored_image_path):
    os.remove(temp_censored_image_path)

# Output the results
print("This video contains nudity. Press 'i' for details.")
input("Press 'i' to view timestamps: ")
for entry in nudity_timestamps:
    print(entry)

print(f"Censored video saved as {censored_output_video}")


In [ ]:
# GPU Code

import ffmpeg
import cv2
from nudenet import NudeDetector
from datetime import timedelta
import os

# Initialize the NudeNet detector
detector = NudeDetector()

# Paths
input_video = "/workspace/GoT/2.mp4"
converted_video = "converted_video.mp4"
censored_output_video = "censored_video.mp4"
temp_image_path = "temp_frame.jpg"  # Temporary file for each frame
temp_censored_image_path = "temp_censored_frame.jpg"  # Temporary censored image path

# Convert to H.265 format with CUDA acceleration and overwriting permission
ffmpeg.input(input_video, hwaccel='cuda').output(converted_video, vcodec='h264_nvenc', crf=28, preset='fast').overwrite_output().run()

# Open the converted video using OpenCV
cap = cv2.VideoCapture(converted_video)
fps = cap.get(cv2.CAP_PROP_FPS)  # Get the original frame rate
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define video writer for the censored video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(censored_output_video, fourcc, fps, (width, height))

# List to store nudity timestamps
nudity_timestamps = []

# List of classes to ignore
ignored_classes = {
    "FEMALE_GENITALIA_COVERED",
    "FACE_FEMALE",
    "BELLY_COVERED",
    "FEET_COVERED",
    "ARMPITS_COVERED",
    "ARMPITS_EXPOSED",
    "BELLY_EXPOSED",
    "ANUS_COVERED",
    "FEMALE_BREAST_COVERED",
    "BUTTOCKS_COVERED",
    "FACE_MALE"
}

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Save the frame as a temporary image file
    cv2.imwrite(temp_image_path, frame)

    # Detect nudity in the saved image
    detections = detector.detect(temp_image_path)

    # Filter detections to exclude ignored classes
    valid_detections = [d for d in detections if d['class'] not in ignored_classes]

    # Sort detections by score, descending order
    valid_detections = sorted(valid_detections, key=lambda x: x['score'], reverse=True)

    # If there are valid detections, apply censoring and log details
    if valid_detections and valid_detections[0]['score'] >= 0.25:  # Threshold check
        # Capture the timestamp and classes with scores
        timestamp = timedelta(seconds=frame_count // fps)
        frame_detections = [f"{d['class']} ({d['score']:.2f})" for d in valid_detections]
        nudity_timestamps.append(f"{str(timestamp)}: " + ", ".join(frame_detections))

        # Censor the frame by applying censoring only to valid detections
        detector.censor(temp_image_path, output_path=temp_censored_image_path)

        # Load the censored frame back into memory
        censored_frame = cv2.imread(temp_censored_image_path)

        # Use the censored frame
        frame = censored_frame

    # Write the frame (censored or not) to the output video
    out.write(frame)
    frame_count += 1

cap.release()
out.release()

# Remove the temporary image files
if os.path.exists(temp_image_path):
    os.remove(temp_image_path)
if os.path.exists(temp_censored_image_path):
    os.remove(temp_censored_image_path)

# Output the results
print("This video contains nudity. Press 'i' for details.")
input("Press 'i' to view timestamps: ")
for entry in nudity_timestamps:
    print(entry)

print(f"Censored video saved as {censored_output_video}")


In [ ]:
# Metrics
import ffmpeg
import cv2
import time
import matplotlib.pyplot as plt

# Define the video paths
input_video = "/workspace/GoT/2.mp4"
converted_video = "converted_video.mp4"

# Function to get processing time for censoring
def get_processing_time(video_path):
    cap = cv2.VideoCapture(video_path)
    start_time = time.time()

    # Example processing: read each frame without censoring
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Simulate processing (here just reading each frame)
        # You could add nudity detection or censoring here as well for full processing time
    cap.release()
    
    return time.time() - start_time

# Convert video using ffmpeg
ffmpeg.input(input_video).output(converted_video, vcodec='libx265', crf=28).overwrite_output().run()

# Measure processing times for both videos
original_processing_time = get_processing_time(input_video)
converted_processing_time = get_processing_time(converted_video)

# Plot the comparison
labels = ['Original Video', 'Converted Video']
times = [original_processing_time, converted_processing_time]

plt.bar(labels, times, color=['blue', 'orange'])
plt.xlabel('Video Type')
plt.ylabel('Processing Time (seconds)')
plt.title('Processing Time Comparison: Original vs. Converted Video')
plt.show()


In [ ]:
# Metrics
import os
import time
import matplotlib.pyplot as plt
import cv2

# Paths to videos
original_video_path = "/workspace/GoT/2.mp4"
converted_video_path = "converted_video.mp4"

# Function to get video file size in MB
def get_file_size(path):
    return os.path.getsize(path) / (1024 * 1024)  # Convert bytes to MB

# Function to calculate bitrate in kbps
def get_bitrate(video_path):
    cap = cv2.VideoCapture(video_path)
    bitrate = cap.get(cv2.CAP_PROP_BITRATE) / 1000  # kbps
    cap.release()
    return bitrate

# Function to get the SSIM comparison for each frame
def calculate_ssim_for_videos(video_path1, video_path2, frame_count=100):
    from skimage.metrics import structural_similarity as ssim
    
    cap1 = cv2.VideoCapture(video_path1)
    cap2 = cv2.VideoCapture(video_path2)
    
    ssim_values = []
    count = 0
    
    while cap1.isOpened() and cap2.isOpened() and count < frame_count:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()
        if not ret1 or not ret2:
            break
        # Convert frames to grayscale for SSIM calculation
        gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        
        # Calculate SSIM between the frames
        ssim_value, _ = ssim(gray1, gray2, full=True)
        ssim_values.append(ssim_value)
        
        count += 1
    
    cap1.release()
    cap2.release()
    
    return sum(ssim_values) / len(ssim_values) if ssim_values else None

# Calculate and store metrics
metrics = {
    "File Size (MB)": [get_file_size(original_video_path), get_file_size(converted_video_path)],
    "Bitrate (kbps)": [get_bitrate(original_video_path), get_bitrate(converted_video_path)]
}

# Processing Time for FFmpeg conversion
start_time = time.time()
ffmpeg.input(original_video_path).output("temp.mp4", vcodec='libx265', crf=28).overwrite_output().run()
processing_time = time.time() - start_time
metrics["Processing Time (s)"] = ["N/A", processing_time]  # Mark original as "N/A" for clarity

# SSIM Calculation
ssim_value = calculate_ssim_for_videos(original_video_path, converted_video_path)
metrics["Average SSIM"] = [1.0, ssim_value]  # 1.0 for original since comparing to itself

# Plotting the metrics
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
fig.suptitle("Comparison of Original and FFmpeg-Converted Video")

# File Size
axs[0, 0].bar(["Original", "Converted"], metrics["File Size (MB)"], color=['blue', 'orange'])
axs[0, 0].set_title("File Size (MB)")

# Bitrate
axs[0, 1].bar(["Original", "Converted"], metrics["Bitrate (kbps)"], color=['blue', 'orange'])
axs[0, 1].set_title("Bitrate (kbps)")

# Processing Time - only show for Converted
axs[1, 0].bar(["Converted"], [metrics["Processing Time (s)"][1]], color=['orange'])
axs[1, 0].set_title("Processing Time (s)")

# SSIM
axs[1, 1].bar(["Original", "Converted"], metrics["Average SSIM"], color=['blue', 'orange'])
axs[1, 1].set_title("Average SSIM")

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()
